# Generalizing a Taylor Recurrence

In [1]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence, recurrence_from_pde, shift_recurrence, get_shifted_recurrence_exp_from_pde, _extract_idx_terms_from_recurrence

import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [2]:
var = _make_sympy_vec("x", 2)
s = sp.Function("s")
g = sp.Function("s")
n = sp.symbols("n")

In [3]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)

w = make_identity_diff_op(2)
helmholtz2d = laplacian(w) + w

In [4]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs

In [5]:
def compute_derivatives_h2d(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    abs_dist = sp.sqrt((var[0]-var_t[0])**2 +
                        (var[1]-var_t[1])**2)
    g_x_y = (1j/4) * hankel1(0, k * abs_dist)
    derivs_helmholtz = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                                                for i in range(p)]
    return derivs_helmholtz

# Step 1: Get recurrence as expression that evaluates to 0 and sanity check it

In [6]:
recur, order = get_shifted_recurrence_exp_from_pde(laplace2d)
order

4

In [7]:
#Sanity check that recurrence is correct
derivs_lap = compute_derivatives(5)
exp = recur.subs(n, 4)
exp.subs(s(4), derivs_lap[4]).subs(s(3), derivs_lap[3]).subs(s(2), derivs_lap[2]).subs(s(1), derivs_lap[1]).subs(var[0],np.random.rand()).subs(var[1],np.random.rand())

4.14251966063262e-15

## Step 2: After performing a Taylor expansion of the $s(n), s(n-1), \dots$ in the 1D recurrence we need to create a 2D grid of coefficients 
$$
grid[i, j]
$$
Where $i = 0$ represents the coefficient attached to $s(n)$ and $i = 1$ represents $s(n-1)$, etc. and the $j$ is for the polynomial in $x_0$.

In [8]:
def get_grid(recur):
    poly_in_s_n = sp.poly(recur, [s(n-i) for i in range(order)])
    coeff_s_n = [poly_in_s_n.coeff_monomial(poly_in_s_n.gens[i]) for i in range(order)]

    table = []
    for i in range(len(coeff_s_n)):
        table.append(sp.poly(coeff_s_n[i], var[0]).all_coeffs()[::-1])

    return table

In [9]:
grid = get_grid(recur)
grid

[[0, (-1)**n*x1**2, 0, (-1)**n],
 [-(-1)**n*n*x1**2 + 3*(-1)**n*x1**2, 0, -3*(-1)**n*n + 5*(-1)**n],
 [0, 3*(-1)**n*n**2 - 13*(-1)**n*n + 14*(-1)**n],
 [-(-1)**n*n**3 + 8*(-1)**n*n**2 - 21*(-1)**n*n + 18*(-1)**n]]

# Step 3: Grid of Coefficient to Grid Recurrence
$$
f(x_1) x_0^k s(n-j) \to f(x_1) x_0^k \sum_{i=0}^{\infty} s_{n-j,i} \frac{x_0^i}{i!} = f(x_1) \sum_{i=k}^{\infty} s_{n-j,i-k} \frac{x_0^i}{(i-k)!} 
$$

In [10]:
def convert(grid):
    recur_exp = 0
    i = sp.symbols("i")
    s_terms = []
    for j in range(len(grid)):
        for k in range(len(grid[j])):
            recur_exp += grid[j][k] * s(n-j,i-k)/sp.factorial(i-k)
            if grid[j][k] != 0:
                s_terms.append((j,k))
    return recur_exp, s_terms
grid_recur, s_terms = convert(grid)
s_terms
grid_recur

(-1)**n*x1**2*s(n, i - 1)/factorial(i - 1) + (-1)**n*s(n, i - 3)/factorial(i - 3) + (-3*(-1)**n*n + 5*(-1)**n)*s(n - 1, i - 2)/factorial(i - 2) + (-(-1)**n*n*x1**2 + 3*(-1)**n*x1**2)*s(n - 1, i)/factorial(i) + (3*(-1)**n*n**2 - 13*(-1)**n*n + 14*(-1)**n)*s(n - 2, i - 1)/factorial(i - 1) + (-(-1)**n*n**3 + 8*(-1)**n*n**2 - 21*(-1)**n*n + 18*(-1)**n)*s(n - 3, i)/factorial(i)

# Step 4: Translate grid recurrence to column recurrence
We can use the fact
$$
s_{n, i} = s_{n-j, i+j} (-1)^j
$$
to perform the following translation:
$$
s_{x, i-l} \to s_{x+l, i} (-1)^l
$$

In [11]:
def grid_recur_to_column_recur(grid_recur, s_terms):
    grid_recur_simp = grid_recur
    bag = set()
    for s_t in s_terms:
        bag.add(-((0-s_t[0])-s_t[1]))
        grid_recur_simp = grid_recur_simp.subs(s(n-s_t[0],i-s_t[1]), (-1)**(s_t[1])*s((n-s_t[0])-s_t[1],(i-s_t[1])+s_t[1]))
    shift = min(bag)
    return sp.solve(sp.simplify(grid_recur_simp * sp.factorial(i)).subs(n, n+shift), s(n,i))[0]

In [12]:
column_recur = grid_recur_to_column_recur(grid_recur, s_terms)
column_recur

NameError: name 'i' is not defined

# Part 5: Package into Big Function:

In [ ]:
def get_taylor_recurrence(pde):
    recur, order = get_shifted_recurrence_exp_from_pde(pde)
    grid = get_grid(recur)
    grid_recur, s_terms = convert(grid)
    column_recur = grid_recur_to_column_recur(grid_recur, s_terms)
    return column_recur

In [ ]:
get_taylor_recurrence(laplace2d)

(-i**2 - 2*i*n + 3*i - n**2 + 3*n - 2)*s(n - 2, i)/x1**2

In [ ]:
get_taylor_recurrence(helmholtz2d)

(-n**3*s(n - 2, 0) + 5*n**2*s(n - 2, 0) - 8*n*s(n - 2, 0) + 4*s(n - 2, 0))/(x1**2*(n - 2))